<p></p>
<br />
<div id="sec_intro_to_intro" style="direction:ltl;line-height:300%;">
	<font face="XB Zar" size=5>
		<font color='crimson' size=7>
            Hypothesis Testing
        </font>
	</font>
</div>

\begin{align*}
H_0 &: \text{Mean of } \textit{X} = \text{Mean of } \textit{Y} \\
H_1 &: \text{Mean of } \textit{X} \neq \text{Mean of } \textit{Y}
\end{align*}



<div id="sec_intro_to_intro" style="direction:ltl;line-height:300%;">
	<font face="XB Zar" size=5>
		<font color='aqua' size=5>
            Importing the Necessary Libraries
        </font>
	</font>
</div>

In [1]:
import pandas as pd
from scipy import stats
from sqlalchemy import create_engine
import plotly.graph_objects as go
from scipy.stats import gaussian_kde
import numpy as np

<div id="sec_intro_to_intro" style="direction:ltl;line-height:300%;">
	<font face="XB Zar" size=5>
		<font color='aqua' size=5>
            Database connection setup
        </font>
	</font>
</div>

In [2]:
username = "root"
password = "root"
server = "localhost"
database_url = f"mysql+mysqlconnector://{username}:{password}@{server}:3306/quera_project_phase_1?charset=utf8mb4"
engine = create_engine(database_url)

<div id="sec_intro_to_intro" style="direction:ltl;line-height:300%;">
	<font face="XB Zar" size=5>
		<font color='aqua' size=5>
            1.What days should I work at Quera?
        </font>
	</font>
</div>

In [3]:
query = """
    SELECT
        c_id,
        date,
        open,
        close
    FROM
        daily_market
"""
df = pd.read_sql_query(query, engine)
df.head()

,c_id,date,open,close
0,1,2023-08-31,27301.9,25931.5
1,1,2023-08-30,27726.1,27297.3
2,1,2023-08-29,26102.5,27727.4
3,1,2023-08-28,26089.6,26106.2
4,1,2023-08-27,26008.2,26089.7


In [4]:
df['price_change'] = abs(df['close'] - df['open'])
df['date'] = pd.to_datetime(df['date'])
df['weekday'] = df['date'].dt.day_name()
df.head()

,c_id,date,open,close,price_change,weekday
0,1,2023-08-31,27301.9,25931.5,1370.4,Thursday
1,1,2023-08-30,27726.1,27297.3,428.8,Wednesday
2,1,2023-08-29,26102.5,27727.4,1624.9,Tuesday
3,1,2023-08-28,26089.6,26106.2,16.6,Monday
4,1,2023-08-27,26008.2,26089.7,81.5,Sunday


In [5]:
weekend_df = df[df['date'].dt.dayofweek.isin([3, 4, 5])]
weekday_df = df[df['date'].dt.dayofweek.isin([6, 0, 1])]
print("Weekend DataFrame:")
print(weekend_df)

print("Weekday DataFrame:")
print(weekday_df)

Weekend DataFrame:
       c_id       date          open         close  price_change   weekday
0         1 2023-08-31  27301.900000  25931.500000   1370.400000  Thursday
5         1 2023-08-26  26047.200000  26008.500000     38.700000  Saturday
6         1 2023-08-25  26163.700000  26047.700000    116.000000    Friday
7         1 2023-08-24  26431.500000  26162.400000    269.100000  Thursday
12        1 2023-08-19  26047.800000  26096.200000     48.400000  Saturday
...     ...        ...           ...           ...           ...       ...
70316   200 2022-09-10      0.005837      0.005934      0.000098  Saturday
70317   200 2022-09-09      0.005661      0.005839      0.000178    Friday
70318   200 2022-09-08      0.005586      0.005655      0.000070  Thursday
70323   200 2022-09-03      0.006022      0.006014      0.000008  Saturday
70324   200 2022-09-02      0.006171      0.006025      0.000146    Friday

[30137 rows x 6 columns]
Weekday DataFrame:
       c_id       date          open

In [6]:
print(weekday_df['price_change'].mean)
print(weekend_df['price_change'].mean)

<bound method NDFrame._add_numeric_operations.<locals>.mean of 2        1624.900000
3          16.600000
4          81.500000
9          99.000000
10         64.600000
            ...     
70314       0.000379
70315       0.000660
70320       0.000432
70321       0.000056
70322       0.000036
Name: price_change, Length: 30136, dtype: float64>
<bound method NDFrame._add_numeric_operations.<locals>.mean of 0        1370.400000
5          38.700000
6         116.000000
7         269.100000
12         48.400000
            ...     
70316       0.000098
70317       0.000178
70318       0.000070
70323       0.000008
70324       0.000146
Name: price_change, Length: 30137, dtype: float64>


In [ ]:
hist = go.Histogram(x=weekend_df['price_change'], opacity=0.75, name="Histogram")

x_vals = np.linspace(min(weekend_df['price_change']), max(weekend_df['price_change']), 4000)
kde = gaussian_kde(weekend_df['price_change'])
y_vals = kde(x_vals)

kde_trace = go.Scatter(x=x_vals, y=y_vals, mode="lines", name="KDE", yaxis="y2")

lower_limit = np.percentile(weekend_df['price_change'], 1)
upper_limit = np.percentile(weekend_df['price_change'], 99)

layout_dual_y = go.Layout(
    title="Histogram and KDE for volume with Dual Y-Axis for other Coins",
    xaxis=dict(title="Value", range=[lower_limit, upper_limit]),
    yaxis=dict(title="Histogram Count"),
    yaxis2=dict(title="KDE Density", overlaying="y", side="right"),
    barmode="overlay",
)

fig_dual_y = go.Figure(data=[hist, kde_trace], layout=layout_dual_y)
fig_dual_y.show()

In [ ]:
hist = go.Histogram(x=weekday_df['price_change'], opacity=0.75, name="Histogram")


x_vals = np.linspace(min(weekday_df['price_change']), max(weekday_df['price_change']), 4000)
kde = gaussian_kde(weekday_df['price_change'])
y_vals = kde(x_vals)

kde_trace = go.Scatter(x=x_vals, y=y_vals, mode="lines", name="KDE", yaxis="y2")

lower_limit = np.percentile(weekday_df['price_change'], 1)
upper_limit = np.percentile(weekday_df['price_change'], 99)

layout_dual_y = go.Layout(
    title="Histogram and KDE for volume with Dual Y-Axis for other Coins",
    xaxis=dict(title="Value", range=[lower_limit, upper_limit]),
    yaxis=dict(title="Histogram Count"),
    yaxis2=dict(title="KDE Density", overlaying="y", side="right"),
    barmode="overlay",
)

fig_dual_y = go.Figure(data=[hist, kde_trace], layout=layout_dual_y)
fig_dual_y.show()

In [ ]:
var_weekend = np.var(weekend_df['price_change'])
var_weekday = np.var(weekday_df['price_change'])

print(var_weekend)
var_weekday

<div id="sec_intro_to_intro" style="direction:ltl;line-height:300%;">
	<font face="XB Zar" size=5>
         From the graphs, it's evident that the distributions aren't normal, making the t-test inapplicable. Instead, we'll employ the Mann-Whitney test, a non-parametric method that doesn't necessitate conditions like normality or equality of variance.
	</font>
</div>

<div id="sec_intro_to_intro" style="direction:ltl;line-height:300%;">
	<font face="XB Zar" size=5>
         We posit that X denotes the weekend days, while Y signifies weekdays.
	</font>
</div>

\begin{align*}
H_0 &: \text{The distribution of } \textit{X} \text{ is the same as the distribution of } \textit{Y} \\
H_1 &: \text{The distribution of } \textit{X} \text{ is different from the distribution of } \textit{Y} \\
& \\
& \text{In other words,} \\
& \\
H_0 &: \text{Median (or Mean) of } \textit{X} = \text{Median (or Mean) of } \textit{Y} \\
H_1 &: \text{Median (or Mean) of } \textit{X} = \text{Median (or Mean) of } \textit{Y}
\end{align*}



<div id="sec_intro_to_intro" style="direction:ltl;line-height:300%;">
	<font face="XB Zar" size=5>
                The Mann-Whitney U test, is chosen because the data does not follow a normal distribution. This non-parametric test compares the distributions of two independent samples without making strict assumptions about the specific distribution from which the data originated.<br>
                We reference the medians when interpreting the results of the Mann-Whitney U test because the median, being a measure of central tendency, accurately reflects the center of a distribution. It remains unaffected by outliers, making it a more reliable measure especially for non-normally distributed data. Though the test doesn't directly pit medians (or means) against each other, a significant outcome indicates a difference in the central tendencies of the two groups.
        </font>
</div>

<div id="sec_intro_to_intro" style="direction:ltl;line-height:300%;">
	<font face="XB Zar" size=5>
		<font color='aqua' size=5>
            Performing Man-Whitney test:
        </font>
	</font>
</div>

In [ ]:
statistic, p_value = stats.mannwhitneyu(weekend_df['price_change'], weekday_df['price_change'])
p_value

<div id="sec_intro_to_intro" style="direction:ltl;line-height:300%;">
	<font face="XB Zar" size=5>
                As evident, the p-value under consideration lacks the decisiveness to reject the null hypothesis outright. Its determination hinges on the threshold we establish for our testing.
        </font>
</div>

<div id="sec_intro_to_intro" style="direction:ltl;line-height:300%;">
	<font face="XB Zar" size=5>
                For instance, when we set the alpha value at 0.01, we lack sufficient evidence to reject the null hypothesis, resulting in its acceptance. In other words, for alpha values less than the p-value, the null hypothesis remains unchallenged (This implies that there is no statistically meaningful difference on the mentioned days, even though various factors such as coin flips and others could be considered. The test results do not reveal any significant distinctions). Conversely, with an alpha of 0.05, we reject the null hypothesis in favor of the alternative hypothesis.
        </font>
</div>